In [1]:
import h5py
from sklearn.metrics import mean_squared_error as mse
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import lightgbm as lgb
import warnings
import os
import xgboost
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn import svm
from sklearn.model_selection import GridSearchCV

from DNN import *
from processing_results import * 
from stacking_models import *

# Warning because some galaxies (those are not of orur interest have mass/ halo concentration 0, which we logarithmize)
warnings.filterwarnings("ignore")

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Example for ELG with n=5e-4, log10[sSFR]=-9.086

In [ ]:
ELGs = pd.read_csv(os.path.join(path, "ELG", "n5e-4_ssfr908.csv"))
train, test = train_test(ELGs, val=False) # change for LRGs

features = ['neigh0.5','neigh1','neigh2','neigh3','neigh4','neigh5','lum_z','lum_r','lum_g','mass','sum_m','sum_V', "anisotropy"]
target = "b_phi" # change to 'halo_conc' to predict halo concentration or 'halo_mass' for halo mass

In [ ]:
# optimized parameters

params = {"xgb": {
    "n_estimators": 500,
    "learning_rate": 0.015,
    "gamma": 2,
    "colsample_bylevel": 0.5,
    "min_child_weight": 0.2,
    "max_depth": 4,
    "subsample": 0.5,
    "reg_lambda": 0.5,
    "alpha": 5,
    "n_jobs": 1,
    "objective": 'reg:squarederror'
},
    "svr": {
        "kernel": 'rbf',
        "C": 5
},
    "DNN": {
        "n_epoch": 120,
        "hidden_layers": 4,
        "hidden_layer_size": 20,
        "dropout": 0.95,
}
}

In [ ]:
stacked_predictions(train[features+["posY"]], train[target], test[features+["posY"]], test[target], params, drop_features=["posY"], batch=10, N=len(features))

# b_phi tertilles 

In [ ]:
# for ELG and LRG predictions

fig,axes=plt.subplots(1,2,figsize=(7,4), constrained_layout=True)

x=[-1,0,1]
x1,x2,_,_=x1_x2(bphi_elg_pred,bphi_elg_act,b1_elg_act)
axes[0].plot(x,x1,"-o",label="Predicted")
axes[0].plot(x,x2,"-o",label="Ideal")

x1,x2,_,_=x1_x2(bphi_lrg_pred,bphi_lrg_act,b1_lrg_act)
axes[1].plot(x,x1,"-o",label="Predicted")
axes[1].plot(x,x2,"-o",label="Ideal")

for i in range(2):
    axes[i].set_xticks((-1,0,1))
    axes[i].set_xlabel("b$_\phi$ tertiles")
    axes[i].set_ylabel("b$_\phi$")
    # axes[i].grid(lw=0.2)
    axes[i].legend()

axes[0].set_title("ELG-like galaxies")
axes[1].set_title("LRG-like galaxies")

# plt.savefig("bphi_results_n5e-4.pdf", bbox_inches='tight', format="pdf")

# Feature importance
check 'processing_results.py' for more

In [ ]:
exp = TreeExplainer(model)
shap_vals = exp.shap_values(test[features])
shap_df = pd.DataFrame(shap_vals, columns=pd.Index(names_to_plot, name='features'))
shap.summary_plot(shap_vals, test[features], plot_type="bar",feature_names=names_to_plot, show=False, plot_size=(4.5,4.5))
plt.xlabel(r"$\overline{|SHAP|}$")
plt.savefig("ELG_importance_real_space.pdf", bbox_inches='tight')
#plt.xlabel(r"$\frac{1}{n}\sum_{i=1}^{n}|{SHAP}_i|$")

In [ ]:
from sklearn.inspection import permutation_importance


perm_importance = permutation_importance(model, test[features], test["b_phi"])
sorted_idx = perm_importance.importances_mean.argsort()
plt.barh(train[features].columns[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

# Plot concentration-mass-bphi results comparison

In [ ]:
# LRGs
fig,axes=plt.subplots(3,1, figsize=(4,7), constrained_layout=True)
#fig.suptitle("LRG-like galaxies")
axes[0].scatter(bphi_lrg_pred, bphi_lrg_act, s=8, label=f"RMSE={round(np.sqrt(mse(bphi_lrg_pred, bphi_lrg_act)),2)}")
axes[1].scatter(conc_lrg_pred, conc_lrg_act, s=8, label=f"RMSE={round(np.sqrt(mse(conc_lrg_pred, conc_lrg_act)),2)}")
axes[2].scatter(mass_lrg_pred, mass_lrg_act, s=8, label=f"RMSE={round(np.sqrt(mse(mass_lrg_pred, mass_lrg_act)),2)}")

axes[0].plot(np.linspace(-2,20,10), np.linspace(-2,20,10), c="r", lw=2)
axes[1].plot(np.linspace(3,13,10), np.linspace(3,13,10), c="r", lw=2)
axes[2].plot(np.linspace(12.7,14.8,10), np.linspace(12.7,14.8,10), c="r", lw=2)

axes[0].set_title("b$_\phi$")
axes[1].set_title("c")

axes[2].set_title("log$_{10}$ M")
for i in range(3):
    # axes[i].grid(lw=0.1)
    axes[i].set_xlabel("Predicted")
    axes[i].set_ylabel("Actual")
    axes[i].legend(loc=4)
    
# plt.savefig("LRG_results.pdf", bbox_inches='tight', format="pdf")

In [ ]:
fig,axes=plt.subplots(3,2, figsize=(8,7), constrained_layout=True)

# ELGs
axes[0,0].scatter(bphi_elg_pred, bphi_elg_act, s=8, label=f"RMSE={round(np.sqrt(mse(bphi_elg_pred, bphi_elg_act)),2)}")
axes[1,0].scatter(conc_elg_pred, conc_elg_act, s=8, label=f"RMSE={round(np.sqrt(mse(conc_elg_pred, conc_elg_act)),2)}")
axes[2,0].scatter(mass_elg_pred, mass_elg_act, s=8, label=f"RMSE={round(np.sqrt(mse(mass_elg_pred, mass_elg_act)),2)}")

axes[0,0].plot(np.linspace(-2,25,10), np.linspace(-2,25,10), c="r", lw=2)
axes[1,0].plot(np.linspace(3,28,10), np.linspace(3,28,10), c="r", lw=2)
axes[2,0].plot(np.linspace(11.5,14.4,10), np.linspace(11.5,14.4,10), c="r", lw=2)

axes[0,0].set_title("ELG, b$_\phi$")
axes[1,0].set_title("ELG, c")
axes[2,0].set_title("ELG, log$_{10}$ M")
for i in range(3):
    #axes[i].grid(lw=0.1)
    axes[i,0].set_xlabel("Predicted")
    axes[i,0].set_ylabel("Actual")
    axes[i,0].legend(loc=4)
    

# LRGs
axes[0,1].scatter(bphi_lrg_pred, bphi_lrg_act, s=8, label=f"RMSE={round(np.sqrt(mse(bphi_lrg_pred, bphi_lrg_act)),2)}")
axes[1,1].scatter(conc_lrg_pred, conc_lrg_act, s=8, label=f"RMSE={round(np.sqrt(mse(conc_lrg_pred, conc_lrg_act)),2)}")
axes[2,1].scatter(mass_lrg_pred, mass_lrg_act, s=8, label=f"RMSE={round(np.sqrt(mse(mass_lrg_pred, mass_lrg_act)),2)}")

axes[0,1].plot(np.linspace(-2,20,10), np.linspace(-2,20,10), c="r", lw=2)
axes[1,1].plot(np.linspace(3,13,10), np.linspace(3,13,10), c="r", lw=2)
axes[2,1].plot(np.linspace(12.7,14.8,10), np.linspace(12.7,14.8,10), c="r", lw=2)

axes[0,1].set_title("LRG, b$_\phi$")
axes[1,1].set_title("LRG, c")
axes[2,1].set_title("LRG, log$_{10}$ M")
for i in range(3):
    #axes[i].grid(lw=0.1)
    axes[i,1].set_xlabel("Predicted")
    axes[i,1].set_ylabel("Actual")
    axes[i,1].legend(loc=4)

plt.savefig("results.pdf", bbox_inches='tight', format="pdf")

# Different samples 

In [ ]:
fig, axes = plt.subplots(1,2,constrained_layout=True)
fig.suptitle("LRGs, redshift space, n=2e-4 with two different sSFR cut",fontweight="bold")
axes[0].set_title("sSFR = 9.23",fontweight="bold")
axes[1].set_title("sSFR = 9.086",fontweight="bold")
axes[0].scatter(predictions_lrg[0], actuals_lrg[0],s=8,label=f"RMSE={round(np.sqrt(mse(predictions_lrg[0], actuals_lrg[0])),2)}")
axes[1].scatter(predictions_lrg[1], actuals_lrg[1],s=8,label=f"RMSE={round(np.sqrt(mse(predictions_lrg[1], actuals_lrg[1])),2)}")

for i in range(2):
    axes[i].grid(lw=0.2)
    axes[i].set_ylabel("Actual")
    axes[i].set_xlabel("Predicted")
    axes[i].legend()
    axes[0].plot(np.linspace(2,14,10),np.linspace(2,14,10),c="r",lw=2.5)

# all samples different selection

In [ ]:
fig,axes=plt.subplots(3,2,figsize=(9,7),constrained_layout=True)
fig.suptitle("ELGs, redshift space",fontweight="bold")
titles = ["log[sSFR] = "]
for i in range(3):
    for j in range(2):
        n=i if j==0 else 3+i
        axes[i,j].set_title(f"log[sSFR]={[-9.08,-9.23][j]}, n={['5e-4', '7e-4', '1e-3'][i]}")
        axes[i,j].scatter(predictions_elg[n], actuals_elg[n],s=5,label=f"RMSE={round(np.sqrt(mse(predictions_elg[n], actuals_elg[n])),2)}")
        axes[i,j].plot(np.linspace(-4,20,10),np.linspace(-4,20,10),c="r",lw=2)
        axes[i,j].grid(lw=0.2)
        axes[i,j].legend()

# Importance 

In [ ]:
# Plot feature importance

def feature_importance(model, train, val, y_val):
    fig, axes = plt.subplots(3, 2, figsize = (20,20))
    fig.suptitle("Feature importance, XGboost", fontweight = "bold")
    
    # BUILD-IN
    # gain
    xgboost_model.importance_type = "gain"
    axes[0,0].set_title("Build-in, 'GAIN'", fontweight = "bold")
    axes[0,0].barh(train.columns, model.feature_importances_)
    # Weight
    xgboost_model.importance_type = "weight"
    axes[1,0].set_title("Build-in, 'WEIGHT'", fontweight = "bold")
    axes[1,0].barh(train.columns, model.feature_importances_)
    # Coverage
    xgboost_model.importance_type = "cover"
    axes[2,0].set_title("Build-in, 'COVER'", fontweight = "bold")
    axes[2,0].barh(train.columns, model.feature_importances_)
    
    # Total gain
    xgboost_model.importance_type = "total_gain"
    axes[2,1].set_title("Build-in, 'Total gain'", fontweight = "bold")
    axes[2,1].barh(train.columns, model.feature_importances_)
    
    # PERMUTATION
    perm_importance = permutation_importance(model, val, y_val)
    #sorted_idx = perm_importance.importances_mean.argsort() # Lets not sort for clarity reasons
    axes[0,1].barh(train.columns, perm_importance.importances_mean)
    axes[0,1].set_title("PERMUTATION importance")
    
    # HEATMAP 
    axes[1,1].set_title("Feature correlations")
    sns.heatmap(train.corr(), vmax=1.0, center=0, fmt='.2f', cmap="YlGnBu",
            square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70}, ax = axes[1,1])

def shap_importance(model, val):    
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(val)
    shap.summary_plot(shap_values, val, plot_type="bar")
    shap.summary_plot(shap_values, X_vaja_val)
    
def save_model(model, filename):
    path = "xgboost_models/" + filename + ".json"
    model.save_model(path)
    
def correlation_heatmap(train):
    correlations = train.corr()

    fig, ax = plt.subplots(figsize=(20,20))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f', cmap="YlGnBu",
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70}
                )
    plt.show()